# Description

This notebook is used to request computation of Water Productivity for customized season period, crop multiplier and area from WaPOR's Gross Biomass Water Productivity (GBWP) or Net Biomass Water Productivity (NBWP) layers using WaPOR API.

You will need WaPOR API Token to use this notebook (See README.md)

# Step 1: Read APIToken

Get your APItoken from https://wapor.apps.fao.org/profile. Enter your API Token when running the cell below.

In [1]:
import requests
import json

path_query=r'https://io.apps.fao.org/gismgr/api/v1/query/'
path_sign_in=r'https://io.apps.fao.org/gismgr/api/v1/iam/sign-in/'

APIToken=input('Your API token: ')

Your API token: Enter Your API token


# Step 2: Get Authorization AccessToken

Using the input API token to get AccessToken for authorization

In [2]:
resp_signin=requests.post(path_sign_in,headers={'X-GISMGR-API-KEY':APIToken})
resp_signin = resp_signin.json()
AccessToken=resp_signin['response']['accessToken']
AccessToken

'eyJhbGciOiJSUzI1NiIsImtpZCI6IjIxODQ1OWJiYTE2NGJiN2I5MWMzMjhmODkxZjBiNTY1M2UzYjM4YmYiLCJ0eXAiOiJKV1QifQ.eyJuYW1lIjoiQmljaCBUcmFuIiwicGljdHVyZSI6Imh0dHBzOi8vbGg0Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tQkxvTDhOYUVLNm8vQUFBQUFBQUFBQUkvQUFBQUFBQUFCTDgvMlFJTzFRQUlxbW8vcGhvdG8uanBnIiwiaXNzIjoiaHR0cHM6Ly9zZWN1cmV0b2tlbi5nb29nbGUuY29tL3VuZmFvLWFwcHMtZGV2IiwiYXVkIjoidW5mYW8tYXBwcy1kZXYiLCJhdXRoX3RpbWUiOjE1OTQ4MTcyMDMsInVzZXJfaWQiOiJSNk5Ibkd5dDRUT3p1M212T2tteE04UkxFMnUxIiwic3ViIjoiUjZOSG5HeXQ0VE96dTNtdk9rbXhNOFJMRTJ1MSIsImlhdCI6MTU5NDgxNzIwMywiZXhwIjoxNTk0ODIwODAzLCJlbWFpbCI6ImJpY2h0bjk0QGdtYWlsLmNvbSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJmaXJlYmFzZSI6eyJpZGVudGl0aWVzIjp7Imdvb2dsZS5jb20iOlsiMTAyMTY0MjA1NjAzMjU1NDMxMjQzIl0sImVtYWlsIjpbImJpY2h0bjk0QGdtYWlsLmNvbSJdfSwic2lnbl9pbl9wcm92aWRlciI6ImN1c3RvbSJ9fQ.w13Mg-FFLhxa2oyoEs_Rr0f7yXMc4ui4IxSoCBDWY10KSMoPasAH4_24MEO_ziEEyAbsIf2QGygI3Xvs-Dd3qa7H5XuVKZqR-IPcPaygNWu9rasuA-33n8X_iSEghDsuJocuJngD2DDpjXPzQ1oa6uneuXEztA8KilQYJ5ir0EoUKffFEGp5fBXUVmvMOmxN-dVYh-VPmETXpMjNd

# Step 3: Write Query Payload

**Arguments** 'waterProductivity' type, 'startSeason', 'endSeason', 'resolution','shape'

Read [Explanatory notes on Custom water productvity](./data/explanatory-notes.pdf) to know how to calculate cropMultiplier

In [3]:
waterProductivity="GBWP" #"GBWP" or "NBWP"
cropMultiplier=0.5
startSeason="2015-01" # "YYYY-DK" (Dekad)
endSeason= "2015-18" # "YYYY-DK" (Dekad)
resolution="100m" #"250m" or "100m" , maybe "30m" works for some area

Define Polygon shape of the area of interest

In [4]:
bbox= [37.95883206252312, 7.89534, 43.32093, 12.3873979377346] #latlon
xmin,ymin,xmax,ymax=bbox[0],bbox[1],bbox[2],bbox[3]
Polygon=[
          [xmin,ymin],
          [xmin,ymax],
          [xmax,ymax],
          [xmax,ymin],
          [xmin,ymin]
        ]

In [5]:
query ={
  "type": "CustomWaterProductivity",
  "params": {
    "waterProductivity": waterProductivity, #"GBWP" or "NBWP"
    "resolution": resolution, #"250m" or "100m" , maybe "30m" works for some area
    "startSeason": startSeason, # "YYYY-DK" (Dekad)
    "endSeason": endSeason, # "YYYY-DK" (Dekad),
    "cropMultiplier":cropMultiplier,
    "shape": {
      "type": "Polygon", #define coordinates of the area in geojson format
      "coordinates": [Polygon]
    }
  }
}

# Step 4: Post the QueryPayload with AccessToken in Header 

In responses, get an url to query job. 

In [6]:
resp_query=requests.post(path_query,headers={'Authorization':'Bearer {0}'.format(AccessToken)},
                                          json=query)
resp_query = resp_query.json()
job_url=resp_query['response']['links'][0]['href']

job_url

'https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/jobs/e32877e8-1431-40e2-ac5e-badd445b8e8d'

# Step 5: Get Job Results.

It will take some time for the job to be finished. When the job is finished, its status will be changed from 'RUNNING' to 'COMPLETED' or 'COMPLETED WITH ERRORS'. If it is COMPLETED, the url to download custom water productivity map can be achieved from Response 'output'.

In [8]:
i=0
print('RUNNING',end=" ")
while i==0:        
    resp = requests.get(job_url)
    resp=resp.json()
    
    if resp['response']['status']=='RUNNING':
        print('.',end =" ")
    if resp['response']['status']=='COMPLETED':
        results=resp['response']['output']
        print('Link to download GBWP',results['bwpDownloadUrl'])
        print('Link to download TBP',results['tbpDownloadUrl'])
        print('Link to download AETI',results['wtrDownloadUrl'])
        i=1
    if resp['response']['status']=='COMPLETED WITH ERRORS':
        print(resp['response']['log'])
        i=1       

RUNNING Link to download GBWP https://io.apps.fao.org/gismgr/download/1ed99653-21e6-4441-b207-7ec9f1cf41a9/L2_GBWP_1501-1518.tif
Link to download TBP https://io.apps.fao.org/gismgr/download/1ed99653-21e6-4441-b207-7ec9f1cf41a9/L2_TBP_1501-1518.tif
Link to download AETI https://io.apps.fao.org/gismgr/download/1ed99653-21e6-4441-b207-7ec9f1cf41a9/L2_AETI_1501-1518.tif
